In [1]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from tqdm import tnrange
import torch.optim as optim
import torch.nn.functional as F
from PROB_HVAE_shGLM import PROB_HVAE_shGLM
from sklearn import metrics
import torch.distributions as dist

# Hyperparams

In [2]:
train_T = 20000
test_T = 8000

C_den = torch.zeros(5,5)
C_den[0,1:] = 1
sub_no = C_den.shape[0]

batch_size = 1500

syn_basis_no = 2
hist_basis_no = 2
T_hist = 201
T_syn = 201
T_enc = 201
hid_dim = 256
temp = 0.5

In [3]:
Ensyn = torch.tensor([0, 106, 213, 211, 99])
Insyn = torch.tensor([1, 22, 36, 42, 19])
E_no = torch.sum(Ensyn)
I_no = torch.sum(Insyn)

C_syn_e = torch.zeros(sub_no, E_no)
C_syn_i = torch.zeros(sub_no, I_no)

E_count = 0
for s in range(sub_no):
    C_syn_e[s,E_count:E_count+Ensyn[s]] = 1
    E_count += Ensyn[s]

I_count = 0
for s in range(sub_no):
    C_syn_i[s,I_count:I_count+Insyn[s]] = 1
    I_count += Insyn[s]

# Train Params

In [4]:
model = PROB_HVAE_shGLM(C_den.cuda(), C_syn_e.cuda(), C_syn_i.cuda(), E_no, T_syn, syn_basis_no,
                T_hist, hist_basis_no, T_enc, hid_dim, temp)
model = model.float().cuda()

V_ref = np.fromfile("/media/hdd01/sklee/cont_shglm/inputs/vdata_NMDA_ApN0.5_13_Adend_r0_o2_i2_g_b0.bin")
V_ref = V_ref[1:-2]

train_V_ref = V_ref[:train_T]
test_V_ref = V_ref[train_T:train_T+test_T]

test_V_ref = torch.from_numpy(test_V_ref).cuda()
train_V_ref = torch.from_numpy(train_V_ref)

In [5]:
E_neural = np.load("/media/hdd01/sklee/cont_shglm/inputs/Espikes_d48000_r1_rep1_Ne629_e5_E20_neural.npy")
I_neural = np.load("/media/hdd01/sklee/cont_shglm/inputs/Ispikes_d48000_r1_rep1_Ni120_i20_I30_neural.npy")

train_S_E = E_neural[:train_T]
train_S_I = I_neural[:train_T]
test_S_E = E_neural[train_T:train_T+test_T]
test_S_I = I_neural[train_T:train_T+test_T]

train_S_E = torch.from_numpy(train_S_E)
train_S_I = torch.from_numpy(train_S_I)
test_S_E = torch.from_numpy(test_S_E).float().cuda()
test_S_I = torch.from_numpy(test_S_I).float().cuda()

In [6]:
repeat_no = 2
batch_no = (train_V_ref.shape[0] - batch_size) * repeat_no
train_idx = np.empty((repeat_no, train_V_ref.shape[0] - batch_size))
for i in range(repeat_no):
    part_idx = np.arange(train_V_ref.shape[0] - batch_size)
    np.random.shuffle(part_idx)
    train_idx[i] = part_idx
train_idx = train_idx.flatten()
train_idx = torch.from_numpy(train_idx)

print(batch_no)
print(train_idx.shape[0])

37000
37000


In [7]:
optimizer = optim.Adam(model.parameters(), lr=0.005)

In [8]:
loss_array = np.empty((batch_no))
beta = 0

for i in tnrange(batch_no):
    
    if i%250 == 249 and i <= 2500:
        beta += 0.1
    
    model.train()
    optimizer.zero_grad()
    batch_idx = train_idx[i].long()
    batch_S_E = train_S_E[batch_idx : batch_idx+batch_size].float().cuda()
    batch_S_I = train_S_I[batch_idx : batch_idx+batch_size].float().cuda()
    batch_ref = train_V_ref[batch_idx:batch_idx+batch_size].float().cuda()
    
    batch_pred, rec_loss, KL_loss, Z_P_prior, Z_P_post = model(batch_ref,
                                                              batch_S_E,
                                                              batch_S_I,
                                                              beta)
        
    loss = rec_loss + beta* KL_loss
    loss_array[i] = loss.item()
    
    print(i, rec_loss.item(), KL_loss.item())
    
    loss.backward()
    optimizer.step()
    
    if i%50 == 0:
        model.eval()
        test_pred, test_Y, test_spikes, test_probs = model.decode(test_S_E, test_S_I)
        test_diff = (test_V_ref - test_pred) ** 1
        test_loss = torch.var(test_diff)
        test_score = metrics.explained_variance_score(y_true=test_V_ref.cpu().detach().numpy(),
                                                      y_pred=test_pred.cpu().detach().numpy(),
                                                      multioutput='uniform_average')
        train_score = metrics.explained_variance_score(y_true=batch_ref.cpu().detach().numpy(),
                                                      y_pred=batch_pred.cpu().detach().numpy(),
                                                      multioutput='uniform_average')
        print("TEST", i, test_loss.item(), test_score.item(), train_score.item())
        print(torch.mean(test_spikes, 0).cpu().detach().numpy())
        #print(torch.mean(spike_probs, 0).cpu().detach().numpy())
        if i%100 == 0:
            torch.save(model.state_dict(), "/media/hdd01/sklee/lvae_shglm/PROB_HVAE_sub5_s2_h2_shglm_i"+str(i)+".pt")
    


<ipython-input-8-71dc3e819a97>:4: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(batch_no):


0 8.096232414245605 0.7123499512672424
TEST 0 13.633274131518244 0.01289063316858896 -0.0026999711990356445
[0.45751572 0.43026686 0.44977623 0.46189693]
1 4.341759204864502 2.2393834590911865
2 nan nan
3 nan nan
4 nan nan
5 nan nan
6 nan nan
7 nan nan
8 nan nan
9 nan nan
10 nan nan
11 nan nan
12 nan nan
13 nan nan
14 nan nan
15 nan nan
16 nan nan
17 nan nan
18 nan nan
19 nan nan
20 nan nan
21 nan nan



KeyboardInterrupt: 

In [ ]:
plt.plot(test_pred.cpu().detach().numpy())

In [ ]:
plt.plot(test_pred.cpu().detach().numpy())